In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, f1_score, matthews_corrcoef,recall_score, precision_score
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split

In [20]:
batch_size = 128
loaded_datasets_info = torch.load('/Users/jiaming/Desktop/Lab2/datas/saved_datasets.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset']
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']

# 1. XGB

In [28]:
from torch.utils.data import DataLoader

def extract_features_labels_from_subset(subset):
    
    loader = DataLoader(subset, batch_size=len(subset))
    
    for features, labels in loader:
        features = features.squeeze(1)
        labels = labels.squeeze(1)
        return features, labels

train_features, train_labels = extract_features_labels_from_subset(loaded_train_dataset)
val_features, val_labels = extract_features_labels_from_subset(loaded_val_dataset)
test_features, test_labels = extract_features_labels_from_subset(loaded_test_dataset)

In [31]:
print(train_features)

tensor([[ 41.,   0., 173.,  ...,   0.,   1.,   0.],
        [ 60.,   0., 280.,  ...,   0.,   1.,   0.],
        [ 51.,  20., 260.,  ...,   1.,   1.,   0.],
        ...,
        [ 51.,   0., 250.,  ...,   0.,   1.,   0.],
        [ 39.,   0., 195.,  ...,   0.,   1.,   0.],
        [ 58.,  20., 206.,  ...,   1.,   1.,   0.]])


In [30]:
print(train_labels.shape)

torch.Size([824])


In [21]:
train_features, train_labels = loaded_train_dataset.tensors
val_features, val_labels = loaded_val_dataset.tensors
test_features, test_labels = loaded_test_dataset.tensors

AttributeError: 'Subset' object has no attribute 'tensors'

In [17]:
X_train = np.array(torch.stack(X_train)).reshape(-1, 25)
y_train = np.array(torch.stack(y_train)).reshape(-1,)

X_test = np.array(torch.stack(X_test)).reshape(-1, 25)
y_test = np.array(torch.stack(y_test)).reshape(-1,)

X_val = np.array(torch.stack(X_val)).reshape(-1, 25)
y_val = np.array(torch.stack(y_val)).reshape(-1,)

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not numpy.ndarray

In [6]:
import xgboost as xgb
import matplotlib.pyplot as plt

In [7]:
# scale
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)
# X_val = sc.transform(X_val)

In [8]:
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',  
    learning_rate=0.1,
    max_depth=10,
    n_estimators=30)

xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=30, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [33]:
y_pred = xgb_model.predict(X_val)
AUC_val = roc_auc_score(y_val, y_pred)
ACC_val = accuracy_score(y_val, (y_pred > 0.5).astype(int))
MCC_val = matthews_corrcoef(y_val, (y_pred > 0.5).astype(int))
Sn_val = recall_score(y_val, (y_pred > 0.5).astype(int))
Sp_val = precision_score(y_val, (y_pred > 0.5).astype(int))
F1_val = f1_score(y_val, (y_pred > 0.5).astype(int))
print(AUC_val, ACC_val,MCC_val,Sn_val,Sp_val, F1_val)

0.6776449491930664 0.8061749571183533 0.16289575591736064 0.1619047619047619 0.40476190476190477 0.23129251700680276


In [34]:
y_pred = xgb_model.predict(X_test)
AUC_test = roc_auc_score(y_test, y_pred)
ACC_test = accuracy_score(y_test, (y_pred > 0.5).astype(int))
MCC_test = matthews_corrcoef(y_test, (y_pred > 0.5).astype(int))
Sn_test = recall_score(y_test, (y_pred > 0.5).astype(int))
Sp_test = precision_score(y_test, (y_pred > 0.5).astype(int))
F1_test = f1_score(y_test, (y_pred > 0.5).astype(int))
print(AUC_test, ACC_test,MCC_test,Sn_test,Sp_test, F1_test)

0.6188899040657834 0.7722908093278463 0.027683806009807653 0.09848484848484848 0.21666666666666667 0.13541666666666666


In [35]:
# 2. Support Vector Machine

In [36]:
X_train = []
y_train = []
for x, y in loaded_train_dataset:
    X_train.append(x)
    y_train.append(y)
X_test = []
y_test = []
for x, y in loaded_test_dataset:
    X_test.append(x)
    y_test.append(y)
X_val = []
y_val = []
for x, y in loaded_val_dataset:
    X_val.append(x)
    y_val.append(y)

In [13]:
X_train = torch.cat(X_train, dim=0).numpy()
y_train = torch.cat(y_train, dim=0).numpy()

X_test = torch.cat(X_test, dim=0).numpy()
y_test = torch.cat(y_test, dim=0).numpy()

X_val = torch.cat(X_val, dim=0).numpy()
y_val = torch.cat(y_val, dim=0).numpy()

In [14]:
from sklearn.svm import SVC
svm_model = SVC(kernel='linear', C=1.0, probability = True)
svm_model.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [15]:
y_pred = svm_model.predict(X_val)
AUC_val = roc_auc_score(y_val, y_pred)
ACC_val = accuracy_score(y_val, (y_pred > 0.5).astype(int))
MCC_val = matthews_corrcoef(y_val, (y_pred > 0.5).astype(int))
Sn_val = recall_score(y_val, (y_pred > 0.5).astype(int))
Sp_val = precision_score(y_val, (y_pred > 0.5).astype(int))
F1_val = f1_score(y_val, (y_pred > 0.5).astype(int))
print(AUC_val, ACC_val,MCC_val,Sn_val,Sp_val, F1_val)

0.5 0.8198970840480274 0.0 0.0 0.0 0.0


/Users/jiaming/Desktop/yes/envs/torch_nightly_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
y_pred = svm_model.predict(X_test)
AUC_test = roc_auc_score(y_test, y_pred)
ACC_test = accuracy_score(y_test, (y_pred > 0.5).astype(int))
MCC_test = matthews_corrcoef(y_test, (y_pred > 0.5).astype(int))
Sn_test = recall_score(y_test, (y_pred > 0.5).astype(int))
Sp_test = precision_score(y_test, (y_pred > 0.5).astype(int))
F1_test = f1_score(y_test, (y_pred > 0.5).astype(int))
print(AUC_test, ACC_test,MCC_test,Sn_test,Sp_test, F1_test)

0.5 0.8189300411522634 0.0 0.0 0.0 0.0


/Users/jiaming/Desktop/yes/envs/torch_nightly_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [191]:
# 3. Random Forest

In [192]:
from sklearn.ensemble import RandomForestClassifier

In [193]:
X_train = []
y_train = []
for x, y in loaded_train_dataset:
    X_train.append(x)
    y_train.append(y)
X_test = []
y_test = []
for x, y in loaded_test_dataset:
    X_test.append(x)
    y_test.append(y)
X_val = []
y_val = []
for x, y in loaded_val_dataset:
    X_val.append(x)
    y_val.append(y)

In [194]:
X_train = np.array(torch.stack(X_train)).reshape(-1, 15)
y_train = np.array(torch.stack(y_train)).reshape(-1,)

X_test = np.array(torch.stack(X_test)).reshape(-1, 15)
y_test = np.array(torch.stack(y_test)).reshape(-1,)

X_val = np.array(torch.stack(X_val)).reshape(-1, 15)
y_val = np.array(torch.stack(y_val)).reshape(-1,)

In [200]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [201]:
y_pred = rf_model.predict(X_val)
AUC_val = roc_auc_score(y_val, y_pred)
ACC_val = accuracy_score(y_val, (y_pred > 0.5).astype(int))
MCC_val = matthews_corrcoef(y_val, (y_pred > 0.5).astype(int))
Sn_val = recall_score(y_val, (y_pred > 0.5).astype(int))
Sp_val = precision_score(y_val, (y_pred > 0.5).astype(int))
F1_val = f1_score(y_val, (y_pred > 0.5).astype(int))
print(AUC_val, ACC_val,MCC_val,Sn_val,Sp_val, F1_val)

0.5869538077403247 0.5865921787709497 0.17535851947783512 0.5222222222222223 0.6025641025641025 0.5595238095238095


In [202]:
y_pred = rf_model.predict(X_test)
AUC_test = roc_auc_score(y_test, y_pred)
ACC_test = accuracy_score(y_test, (y_pred > 0.5).astype(int))
MCC_test = matthews_corrcoef(y_test, (y_pred > 0.5).astype(int))
Sn_test = recall_score(y_test, (y_pred > 0.5).astype(int))
Sp_test = precision_score(y_test, (y_pred > 0.5).astype(int))
F1_test = f1_score(y_test, (y_pred > 0.5).astype(int))
print(AUC_test, ACC_test,MCC_test,Sn_test,Sp_test, F1_test)

0.6030882115787777 0.6053811659192825 0.2070295360126553 0.5566037735849056 0.59 0.5728155339805824


In [203]:
# 4. Naive Bayes

In [204]:
from sklearn.naive_bayes import GaussianNB

In [205]:
X_train = []
y_train = []
for x, y in loaded_train_dataset:
    X_train.append(x)
    y_train.append(y)
X_test = []
y_test = []
for x, y in loaded_test_dataset:
    X_test.append(x)
    y_test.append(y)
X_val = []
y_val = []
for x, y in loaded_val_dataset:
    X_val.append(x)
    y_val.append(y)

In [206]:
X_train = np.array(torch.stack(X_train)).reshape(-1, 15)
y_train = np.array(torch.stack(y_train)).reshape(-1,)

X_test = np.array(torch.stack(X_test)).reshape(-1, 15)
y_test = np.array(torch.stack(y_test)).reshape(-1,)

X_val = np.array(torch.stack(X_val)).reshape(-1, 15)
y_val = np.array(torch.stack(y_val)).reshape(-1,)

In [207]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB()

In [208]:
y_pred = nb_model.predict(X_val)
AUC_val = roc_auc_score(y_val, y_pred)
ACC_val = accuracy_score(y_val, (y_pred > 0.5).astype(int))
MCC_val = matthews_corrcoef(y_val, (y_pred > 0.5).astype(int))
Sn_val = recall_score(y_val, (y_pred > 0.5).astype(int))
Sp_val = precision_score(y_val, (y_pred > 0.5).astype(int))
F1_val = f1_score(y_val, (y_pred > 0.5).astype(int))
print(AUC_val, ACC_val,MCC_val,Sn_val,Sp_val, F1_val)

0.599438202247191 0.5977653631284916 0.24807342837609303 0.3 0.75 0.4285714285714285


In [210]:
y_pred = nb_model.predict(X_test)
AUC_test = roc_auc_score(y_test, y_pred)
ACC_test = accuracy_score(y_test, (y_pred > 0.5).astype(int))
MCC_test = matthews_corrcoef(y_test, (y_pred > 0.5).astype(int))
Sn_test = recall_score(y_test, (y_pred > 0.5).astype(int))
Sp_test = precision_score(y_test, (y_pred > 0.5).astype(int))
F1_test = f1_score(y_test, (y_pred > 0.5).astype(int))
print(AUC_test, ACC_test,MCC_test,Sn_test,Sp_test, F1_test)

0.6032091598129333 0.6143497757847534 0.23244739014736424 0.37735849056603776 0.6666666666666666 0.4819277108433735


In [ ]:
# 5. Logistic Regresssion

In [221]:
from sklearn.linear_model import LogisticRegression

In [222]:
X_train = []
y_train = []
for x, y in loaded_train_dataset:
    X_train.append(x)
    y_train.append(y)
X_test = []
y_test = []
for x, y in loaded_test_dataset:
    X_test.append(x)
    y_test.append(y)
X_val = []
y_val = []
for x, y in loaded_val_dataset:
    X_val.append(x)
    y_val.append(y)

In [223]:
X_train = np.array(torch.stack(X_train)).reshape(-1, 15)
y_train = np.array(torch.stack(y_train)).reshape(-1,)

X_test = np.array(torch.stack(X_test)).reshape(-1, 15)
y_test = np.array(torch.stack(y_test)).reshape(-1,)

X_val = np.array(torch.stack(X_val)).reshape(-1, 15)
y_val = np.array(torch.stack(y_val)).reshape(-1,)

In [230]:
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)

/Users/jiaming/Desktop/yes/envs/torch_nightly_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

In [231]:
y_pred = lr_model.predict(X_val)
AUC_val = roc_auc_score(y_val, y_pred)
ACC_val = accuracy_score(y_val, (y_pred > 0.5).astype(int))
MCC_val = matthews_corrcoef(y_val, (y_pred > 0.5).astype(int))
Sn_val = recall_score(y_val, (y_pred > 0.5).astype(int))
Sp_val = precision_score(y_val, (y_pred > 0.5).astype(int))
F1_val = f1_score(y_val, (y_pred > 0.5).astype(int))
print(AUC_val, ACC_val,MCC_val,Sn_val,Sp_val, F1_val)

0.6313982521847691 0.6312849162011173 0.26299357714141824 0.6111111111111112 0.6395348837209303 0.6250000000000001


In [232]:
y_pred = lr_model.predict(X_test)
AUC_test = roc_auc_score(y_test, y_pred)
ACC_test = accuracy_score(y_test, (y_pred > 0.5).astype(int))
MCC_test = matthews_corrcoef(y_test, (y_pred > 0.5).astype(int))
Sn_test = recall_score(y_test, (y_pred > 0.5).astype(int))
Sp_test = precision_score(y_test, (y_pred > 0.5).astype(int))
F1_test = f1_score(y_test, (y_pred > 0.5).astype(int))
print(AUC_test, ACC_test,MCC_test,Sn_test,Sp_test, F1_test)

0.6331640058055152 0.6322869955156951 0.26607068980059945 0.6509433962264151 0.6052631578947368 0.6272727272727273
